# Stability calculations

## -- LUVOIR --

In notebook 10, we calculated the PASTIS modes for all three currently designed apodizers for the LUVOIR A pupil. In this notebook, we will load the modes and calculate the maximum mode contributions $\sigma$ and strability requirements $\Delta \sigma$.

After that I also calculate the maximum segment contributions $\mu$.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging_new import LuvoirAPLC

In [ ]:
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Computer Modern Sans"]
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 16

## Instantiate a simple SM for mode plotting

I need to simplify this... I can make a child class like I did for the full LUVOIR simulator.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(hc.UnstructuredCoords(poslist))

## Mode and eigenvalue units

Since the matrices got generated normalized to an aberration of 1 nm, everything stemming from the matrix will be in units of **nanometers**. --> This is the desired behaviour of course and is yet to be confirmed, which is one of the goals of this notebook.

In [ ]:
eunit = 1e-9

## Also instantiate a LUVOR telescope

For cumulative contrast calculation.

> **ATTENTION:** the variable `apodizer_design` has to be correct for the data we are reading with the variable `savedpath` further below!!!

In [ ]:
savedpath = '/Users/pueyo/Documents/data_from_repos/pastis_data/2020-06-22T14-01-35_luvoir-medium'   # large apodizer LUVOIR
matrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_Multiple_Modes_Harris_Thermal.fits'))
plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
Pad_orients = np.pi / 2 * np.ones(120)
harris_file = '/Users/pueyo/Desktop/ULTRA/HarrisData/Sensitivities2.xlsx'
luvoir.make_segment_Harris_thermal_primary(harris_file, Pad_orients)

In [ ]:
zernike_coeffs = np.zeros([5*120])
zernike_coeffs[4] = eunit
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_sm.phase)

In [ ]:
zernike_coeffs = np.zeros([5*120])
zernike_coeffs[4] = 0
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()
plt.figure(figsize=(10, 10))
hc.imshow_field(psf_unaber, norm=LogNorm())

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', baseline_contrast)

## Read eigenmodes and eigenvalues

In [ ]:
evals, evecs = np.linalg.eig(matrix)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]
plt.figure(figsize=(14, 8))
#plt.plot(evals, label='Unsorted from eigendecomposition')
plt.plot(sorted_evals, label='Sorted lowest to highest evals')
plt.semilogy()
plt.xlabel('Eigenmodes')
plt.ylabel('Log Eigenvalues')
plt.legend()

In [ ]:
# Lets compute all modes now for LUVOIR with an HCIPy SM
emodes = []

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    zernike_coeffs = eunit*sorted_evecs[:, mode]/2
    luvoir.sm.actuators = zernike_coeffs
    wf_sm = luvoir.sm(luvoir.wf_aper)
    emodes.append(wf_sm.phase)

## Segment based constraints

In [ ]:
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Computer Modern Sans"]
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 16

In [ ]:
c_target = 1e-10

In [ ]:
number_of_modes = luvoir.sm.num_actuators
# Calculate the inverse of the pastis MODE matrix
modestosegs = np.linalg.pinv(sorted_evecs)

# # Calculate all mean contrasts of the pastis modes directly (as-is, with natural normalization)
# c_avg = []
# for i in range(number_of_modes):
#     c_avg.append(util.pastis_contrast(sorted_evecs[:, i] * u.nm, matrix) + baseline_contrast)

# # Calculate segment requirements
# mu_map = np.sqrt(
#     ((c_target - baseline_contrast) / number_of_modes) / (np.dot(c_avg - baseline_contrast, np.square(modestosegs))))

# Calculate all mean contrasts of the pastis modes directly (as-is, with natural normalization)
c_avg = []
for i in range(number_of_modes):
    c_avg.append(util.pastis_contrast(sorted_evecs[:, i] * u.nm, matrix) )

# Calculate segment requirements
mu_map = np.sqrt(
    ((c_target ) / number_of_modes) / (np.dot(c_avg, np.square(modestosegs))))

In [ ]:
zernike_coeffs = np.zeros([5*120])
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
psf_unaber, ref, inter_unaber = luvoir.calc_psf(ref=True, return_intermediate='efield')

In [ ]:
zernike_coeffs = mu_map*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf.intensity/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = np.abs(psf.electric_field - psf_unaber.electric_field)**2/norm 
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', test_contrast)

In [ ]:
N_zernike = 5
zernike_coeffs_numaps_Drifts = np.zeros([N_zernike,120])
for qq in range(N_zernike):
    zernike_coeffs_tmp = np.zeros([120])
    for pp in range(120):
        zernike_coeffs_tmp[pp] = mu_map[qq+(pp)*N_zernike]
    zernike_coeffs_numaps_Drifts[qq] = zernike_coeffs_tmp

In [ ]:
N_zernike = 5
zernike_coeffs_numaps = np.zeros([N_zernike,number_of_modes])
for qq in range(N_zernike):
    zernike_coeffs_tmp = np.zeros([number_of_modes])
    for pp in range(120):
        zernike_coeffs_tmp[qq+(pp)*N_zernike] = mu_map[qq+(pp)*N_zernike]
    zernike_coeffs_numaps[qq] = zernike_coeffs_tmp

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
luvoirDUMMY = LuvoirAPLC(optics_input, apodizer_design, sampling)
luvoirDUMMY.make_segment_zernike_primary(1)

In [ ]:
luvoirDUMMY.wf_aper.wavenumber*eunit

In [ ]:
nu_maps = []
for qq in range(N_zernike):
    zernike_coeffs = eunit*zernike_coeffs_numaps[qq]/2
    luvoir.sm.actuators = zernike_coeffs
    nu_maps.append(luvoir.sm(luvoir.wf_aper).phase/luvoir.wf_aper.wavenumber)

In [ ]:
nu_maps_Drifts = []
for qq in range(N_zernike):
    zernike_coeffs = eunit*zernike_coeffs_numaps_Drifts[qq]/2
    luvoirDUMMY.sm.actuators = zernike_coeffs
    nu_maps_Drifts.append(luvoirDUMMY.sm(luvoirDUMMY.wf_aper).phase/luvoirDUMMY.wf_aper.wavenumber/eunit)

In [ ]:
zernike_coeffs = np.zeros([120])
zernike_coeffs[3] = eunit/2
luvoirDUMMY.sm.actuators = zernike_coeffs
tost = luvoirDUMMY.sm(luvoirDUMMY.wf_aper).phase/luvoirDUMMY.wf_aper.wavenumber/eunit
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(tost, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment')
zernike_coeffs = np.zeros([5*120])
zernike_coeffs[0] = eunit/2
luvoir.sm.actuators = zernike_coeffs
tast = luvoir.sm(luvoir.wf_aper).phase/luvoir.wf_aper.wavenumber/eunit
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(tast, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment')

In [ ]:
# zernike_coeffs = np.zeros([5*120])
# zernike_coeffs[0] = 1e-12/2
# luvoir.sm.actuators = zernike_coeffs
# tast = luvoir.sm(luvoir.wf_aper).phase/luvoir.wf_aper.wavenumber/1e-12
# fig, ax = plt.subplots(ncols=1)
# img1 = hc.imshow_field(tast, cmap='RdBu')#, vmin=pmin, vmax=pmax)
# clb = fig.colorbar(img1)
# clb.set_label('pm', rotation=90)
# plt.tight_layout(h_pad=1)
# plt.title('Segment')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[0]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk Faceplates Silvered ')
zernike_coeffs = zernike_coeffs_numaps[0]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])- baseline_contrast
print('contrast:', test_contrast)
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps_Drifts[0]*1000,cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('mk', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Faceplates Silvered ')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[1]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk bulk ')
zernike_coeffs = zernike_coeffs_numaps[1]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps_Drifts[1]*1000,cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('mk', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Bulk')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[2]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradiant radial')
zernike_coeffs = zernike_coeffs_numaps[2]*eunit / 2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps_Drifts[2]*1000,cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('mk', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Gradiant radial')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[3]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradient X lateral  ')
zernike_coeffs = zernike_coeffs_numaps[3]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps_Drifts[3]*1000,cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('mk', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Gradient X lateral')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[4]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradient Z axial')
zernike_coeffs = zernike_coeffs_numaps[3]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps_Drifts[4]*1000,cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('mk', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Gradient Z axial')

# Drawing the modes

In [ ]:
import pandas as pd 
from scipy.interpolate import griddata

In [ ]:
df = pd.read_excel(harris_file)
#Thermal: a, h, i, j, k  
#Mechnical: e, f, g
#Other: b, c, d
X = np.asarray(df.X)
Y = np.asarray(df.Y)
A = np.asarray(df.a)
B = np.asarray(df.b)
C = np.asarray(df.c)
D = np.asarray(df.d)
E = np.asarray(df.e)
F = np.asarray(df.f)
G = np.asarray(df.g)
H = np.asarray(df.h)
I = np.asarray(df.i)
J = np.asarray(df.j)
K = np.asarray(df.k)
# ti = np.linspace(-1,1,200)
# XI, YI = np.meshgrid(ti, ti)
# points = np.transpose(np.asarray([X,Y]))
# values = A
# ZI = griddata(points, values, (XI, YI), method='linear')
# f = plt.figure()
# plt.imshow(ZI)
# plt.clim(min(A),max(A))
# plt.colorbar()
# phi = np.pi/2
# XRot = XI * np.cos(phi) + YI * np.sin(phi)
# YRot = -XI * np.sin(phi) + YI * np.cos(phi)
# pointsR = np.transpose(np.asarray([X,Y]))
# values = A
# ZI = griddata(pointsR, values, (XRot, YRot), method='linear')
# f = plt.figure()
# plt.imshow(ZI)
# plt.clim(min(A),max(A))
# plt.colorbar()
X = np.asarray(df.X)
Y = np.asarray(df.Y)
HarrisDiam = np.max([np.max(X)-np.min(X),np.max(Y)-np.min(Y)])
pup_dims = luvoir.pupil_grid.dims
X = np.asarray(df.X)*luvoir.diam/HarrisDiam
Y = np.asarray(df.Y)*luvoir.diam/HarrisDiam
ti = np.linspace(-0.5,0.5,pup_dims[0])*luvoir.diam
points = np.transpose(np.asarray([X,Y]))
valuesA = A
valuesB = B
valuesC = C
valuesD = D
valuesE = E
valuesF = F
valuesG = G
valuesH = H
valuesI = I
valuesJ = J
valuesK = K
XI, YI = np.meshgrid(ti, ti)
# ZI = griddata(points, valuesA, (XI, YI), method='linear')
# ZI[np.isnan(ZI)]=0
# f = plt.figure()
# plt.imshow(ZI)
# plt.clim(min(A),max(A))
# plt.colorbar()
phi = np.pi/2
pointsR = np.transpose(np.asarray([X,Y]))
XRot = XI * np.cos(phi) + YI * np.sin(phi)
YRot = -XI * np.sin(phi) + YI * np.cos(phi)
ZA = griddata(points, valuesA, (XRot, YRot), method='linear')
ZA[np.isnan(ZA)]=0
# ZA = ZA.ravel()
ZH = griddata(points, valuesH, (XRot, YRot), method='linear')
ZH[np.isnan(ZH)]=0
# ZH = ZH.ravel()
ZI = griddata(points, valuesI, (XRot, YRot), method='linear')
ZI[np.isnan(ZI)]=0
# ZI = ZI.ravel()
ZJ = griddata(points, valuesJ, (XRot, YRot), method='linear')
ZJ[np.isnan(ZJ)]=0
# ZJ = ZJ.ravel()
ZK = griddata(points, valuesK, (XRot, YRot), method='linear')
# ZK[np.isnan(ZK)]=0
fig, ax = plt.subplots(ncols=1)
img1 = plt.imshow(ZA, cmap='RdBu')#, vmin=pmin, vmax=pmax)
plt.axis('off')
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk Faceplates Silvered')
fig, ax = plt.subplots(ncols=1)
img2 = plt.imshow(ZH, cmap='RdBu')#, vmin=pmin, vmax=pmax)
plt.axis('off')
clb = fig.colorbar(img2)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk bulk')
fig, ax = plt.subplots(ncols=1)
img3 = plt.imshow(ZI, cmap='RdBu')#, vmin=pmin, vmax=pmax)
plt.axis('off')
clb = fig.colorbar(img3)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradiant radial')
fig, ax = plt.subplots(ncols=1)
img4 = plt.imshow(ZJ, cmap='RdBu')#, vmin=pmin, vmax=pmax)
plt.axis('off')
clb = fig.colorbar(img4)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradient X lateral')
fig, ax = plt.subplots(ncols=1)
img5 = plt.imshow(ZK, cmap='RdBu')#, vmin=pmin, vmax=pmax)
plt.axis('off')
clb = fig.colorbar(img5)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level 1mk gradient Z axial')

# E2E histogram

In [ ]:
n_repeat = 3000
all_contr_rand_seg = []
all_random_maps = []
for rep in range(n_repeat):
    print('Segment realization {}/{}'.format(rep + 1, n_repeat))
    #zernike_coeffs = np.random.uniform(-mu_map*eunit,mu_map*eunit,number_of_modes)
    zernike_coeffs = np.random.uniform(0,mu_map*eunit,number_of_modes)
    luvoir.sm.actuators = zernike_coeffs
    luvoir.sm(luvoir.wf_aper)
    psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')
#     plt.figure(figsize=(10, 10))
#     hc.imshow_field(psf.intensity/norm, norm=LogNorm())
#     plt.colorbar()
    dh_intensity = np.abs(psf.electric_field - psf_unaber.electric_field)**2/norm 
    test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
    all_contr_rand_seg.append(test_contrast) 
    print('contrast:', test_contrast)
    

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(all_contr_rand_seg,30)
plt.title('E2E raw contrast, {} realizations, target contrast 1e-10'.format(n_repeat), size=20)
plt.xlabel('Mean contrast in DH', size=20)
plt.ylabel('PDF', size=20)
plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=25)

In [ ]:
x